In [5]:
from nltk.translate.bleu_score import SmoothingFunction
from bert_score import score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import evaluate
rouge = evaluate.load('rouge')
import random
random.seed(0)

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

import torch
def generate_content(tokenizer, messages):
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    tokenizer.pad_token = tokenizer.eos_token
    with torch.no_grad():
        outputs = model.generate(
                input_ids=input_ids,
                pad_token_id=tokenizer.eos_token_id,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
        )
        
        val = (tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("[/INST]")[-1])
    
    return val, input_ids

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/csgrad/sunilruf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ['TRANSFORMERS_CACHE'] = '/home/csgrad/sunilruf/'
#path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/rank_64/rank_64_2/checkpoint-50"
#path = "/data/sunilruf/prune/wanda/out/llama2_7b/model"
#path = "/data/sunilruf/llama/phi3/checkpoint-49/"
#path = "/data/sunilruf/llama/fp16_no/rank_32_1/checkpoint-50/"
path = "microsoft/Phi-3-mini-4k-instruct/"
torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True, device_map="auto")

OSError: Incorrect path_or_model_id: 'microsoft/Phi-3-mini-4k-instruct/'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [2]:
from datasets import load_dataset

dataset = load_dataset("thu-coai/esconv")

In [3]:
import json
data = json.loads(dataset['test'][5]['text'])
data = data['dialog']
print(data[0]['text'])

My friends are trying to shame me.


In [4]:
val = 0
import json
for i in range(3,10):
    data = json.loads(dataset['test'][i]['text'])
    val += (len(data['dialog']))

print(val)

271


In [6]:
import torch
from tqdm import tqdm
import numpy as np
import time
import json
start_time = time.time()

max_length = 1024
stride = 128
smoothie = SmoothingFunction().method4
#encodings = tokenizer.encode_plus(data['dialog'][1]['text'], return_tensors="pt").to('cuda')
tokenizer.pad_token = tokenizer.eos_token
seq_len = 1024


final_nlls = []

final_nlls_boh = []
prev_end_loc = 0
messages = []
i = 1
final_messages = []
final_messages_boh = []
final_messages_boh1 = []
final_perplexity = []
final_perplexity_boh = []
bleu_final, bleu_boh_final = [], []
rouge_1_final, rouge_2_final, rougle_l_final = [], [], []
Bert_P_final, Bert_R_final, Bert_F1_final = [], [], []
rouge_1_boh_final, rouge_2_boh_final, rougle_l_boh_final = [], [], []
Bert_P_boh_final, Bert_R_boh_final, Bert_F1_boh_final = [], [], []
output_len_final, output_len_boh_final = [], []
for j in range(10):
    #num = random.choice(range(3000, 3110))
    num = j
    #data = extes_data[num]['content']
    data = json.loads(dataset['test'][num]['text'])
    data = data['dialog']
    print(data)
    perplexity = []
    perplexity_boh = []
    nlls = []
    nlls_boh = []
    #data = data
    messages = []
    messages_boh = []
    messages_boh1 = []
    bleu_1, bleu_1_boh = [], []
    rouge_1, rouge_2, rougle_l = [], [], []
    Bert_P, Bert_R, Bert_F1 = [], [], []
    rouge_1_boh, rouge_2_boh, rougle_l_boh = [], [], []
    Bert_P_boh, Bert_R_boh, Bert_F1_boh = [], [], []
    output_len, output_len_boh = [], []
    try:
        for i in range(0, len(data), 1):
            for begin_loc in (range(0, seq_len, stride)):
                #print(data[i])
                
                if data[i]['speaker'] == 'sys':
                    break
                    
                #print(data[i]['User'])
                #print(data[i+1]['AI'])
                messages.append({"role": "user", "content":  data[i]['text']})
                print(messages)
                messages_boh.append({"role": "user", "content":  remove_stopwords(data[i]['text'])})
                messages_boh1.append({"role": "user", "content":  data[i]['text']})
                print(messages)
                #i+=1
                end_loc = min(begin_loc + max_length, seq_len)
                trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
                #input_ids = encodings.input_ids[:, begin_loc:end_loc].to('cuda')
                input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
                input_ids_boh = tokenizer.apply_chat_template(messages_boh, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
                target_ids = input_ids.clone()
                target_ids_boh = input_ids_boh.clone()
                target_ids[:, :-trg_len] = -100
                target_ids_boh[:, :-trg_len] = -100

                with torch.no_grad():
                    
                    
                    val, _ = generate_content(tokenizer, messages)
                    val_boh, _ = generate_content(tokenizer, messages_boh)
                    candidate = val
                    candidate_boh = val_boh
                    if data[i+1]['speaker'] == "sys":
                        reference = data[i+1]['text']
                    elif data[i+2]['speaker'] == "sys":
                        reference = data[i+1]['text']
                    elif data[i+3]['speaker'] == "sys":
                        reference = data[i+1]['text']
                        
                    references = reference.split(" ")
                    candidates = val.split(" ")
                    candidates_boh = val_boh.split(" ")
                    outputs = model(input_ids, labels=target_ids)
                    print(val)
                    print(val_boh)
                    messages.append({"role": "assistant", "content": val})
                    print(outputs.logits.shape)
                    output_len.append(outputs.logits.shape[1])
                    outputs_boh = model(input_ids_boh, labels=target_ids_boh)
                    print(outputs_boh.logits.shape)
                    output_len_boh.append(outputs_boh.logits.shape[1])
                    messages_boh.append({"role": "assistant", "content": remove_stopwords(val_boh)})
                    messages_boh1.append({"role": "assistant", "content": val_boh})
                        
                    
                    neg_log_likelihood = outputs.loss
                    neg_log_likelihood_boh = outputs_boh.loss
                    
                    bleu_1.append(sentence_bleu(references, candidates, weights=(1, 0, 0, 0), smoothing_function=smoothie))
                    results = rouge.compute(predictions=[candidate], references=[reference])
                    rouge_1.append(results['rouge1'])
                    rouge_2.append(results['rouge2'])
                    rougle_l.append(results['rougeL'])
                    
                    P, R, F1 = score([candidate], [reference], lang='en', model_type='bert-base-uncased')
                    Bert_P.append(P)
                    Bert_R.append(R)
                    Bert_F1.append(F1)
                    
                    bleu_1_boh.append(sentence_bleu(references, candidates_boh, weights=(1, 0, 0, 0), smoothing_function=smoothie))
                    results_boh = rouge.compute(predictions=[candidate_boh], references=[reference])
                    rouge_1_boh.append(results_boh['rouge1'])
                    rouge_2_boh.append(results_boh['rouge2'])
                    rougle_l_boh.append(results_boh['rougeL'])
                    
                    P, R, F1 = score([candidate_boh], [reference], lang='en', model_type='bert-base-uncased')
                    Bert_P_boh.append(P)
                    Bert_R_boh.append(R)
                    Bert_F1_boh.append(F1)
                    #print(neg_log_likelihood.shape)

                nlls.append(neg_log_likelihood)
                nlls_boh.append(neg_log_likelihood_boh)
                #print(len(nlls))
                prev_end_loc = end_loc
                if end_loc == seq_len:
                    break

            ppl = torch.exp(torch.stack(nlls).mean())
            ppl_boh = torch.exp(torch.stack(nlls_boh).mean())
            print(ppl)
            print(ppl_boh)
            perplexity.append(ppl)
            perplexity_boh.append(ppl_boh)
            
    except Exception as e:
        print(e)
    
    final_perplexity.append(perplexity)
    final_perplexity_boh.append(perplexity_boh)
    final_nlls.append(nlls)
    final_nlls_boh.append(nlls_boh)
    final_messages.append(messages)
    final_messages_boh.append(messages_boh)
    final_messages_boh1.append(messages_boh1)
    bleu_final.append(bleu_1)
    rouge_1_final.append(rouge_1)
    rouge_2_final.append(rouge_2)
    rougle_l_final.append(rougle_l)
    Bert_P_final.append(Bert_P)
    Bert_R_final.append(Bert_R)
    Bert_F1_final.append(Bert_F1)
    bleu_boh_final.append(bleu_1_boh)
    rouge_1_boh_final.append(rouge_1_boh)
    rouge_2_boh_final.append(rouge_2_boh)
    rougle_l_boh_final.append(rougle_l_boh)
    Bert_P_boh_final.append(Bert_P_boh)
    Bert_R_boh_final.append(Bert_R_boh)
    Bert_F1_boh_final.append(Bert_F1_boh)
    output_len_final.append(output_len)
    output_len_boh_final.append(output_len_boh)

end_time = time.time()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'text': 'Hello. How are you today?', 'speaker': 'sys', 'strategy': 'Others'}, {'text': 'hi i am okay, a little bit sad though', 'speaker': 'usr'}, {'text': 'Okay. I am very sorry to hear that! Do you want to tell me more about that?', 'speaker': 'sys', 'strategy': 'Question'}, {'text': 'Well with the holidays coming up i have been very stressed and nervous about what i am going to do', 'speaker': 'usr'}, {'text': 'I really feel you there. Holidays are so hard.. especially with the way this year has been. Anything specific?', 'speaker': 'sys', 'strategy': 'Affirmation and Reassurance'}, {'text': "Yes. I am a college student and I live on campus. The day before I moved in my dad said some unforgivable things to me and we haven't spoke since", 'speaker': 'usr'}, {'text': 'I have no choice but to go live at home over the break but I am very scared about it', 'speaker': 'usr'}, {'text': "Do you feel scared for your own self? Or of the situation in general? I'm sorry to hear that. Relation

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


The size of tensor a (4096) must match the size of tensor b (4097) at non-singleton dimension 3


In [7]:
#end_time = time.time()

In [7]:
print(end_time - start_time)

2665.9538383483887


In [12]:
print(output_len_boh_final)
print(output_len_final)

[[], [], [], [5, 43, 181, 436, 996, 2119, 4096], [], [9, 75, 197, 472, 1070, 2265, 4096], [5, 17, 37, 78, 205, 511, 1132, 2377, 4096], [], [], [6, 19, 59, 210, 544, 1210, 2545, 4096]]
[[], [], [], [5, 76, 331, 793, 1748, 3692, 4096], [], [12, 119, 306, 757, 1645, 3418, 4096], [5, 24, 115, 327, 779, 1679, 3469, 4096, 4096], [], [], [6, 23, 80, 245, 615, 1359, 2860, 4096]]


In [8]:
def compute_average(value):
    value2 = np.array([])
    for val in value:
        #print(val)
        val = np.array(val)
        #print(val)
        #print(np.mean(val, axis=0))
        value2 = np.append(value2, np.mean(val, axis=0))
    #print(value2)
    return np.mean(value2)
        
def compute_bert(value):
    value2 = np.array([])
    for val in value:
        #print(val)
        val = np.array([tensor.item() for tensor in val])
        #print(val)
        #print(np.mean(val, axis=0))
        value2 = np.append(value2, np.mean(val, axis=0))
    #print(value2)
    return np.mean(value2)
def calc_out(value):
    try:
        value2 = []
        for i in value:
            value2.append(i[-1])
    except:
        pass
    return value2

In [9]:
import numpy as np

rouge1 = compute_average([x for x in rouge_1_final if x])
rouge2 = compute_average([x for x in rouge_2_final if x])
rougel = compute_average([x for x in rougle_l_final if x])
bleu = compute_average([x for x in bleu_final if x])
bert_p = compute_bert([x for x in Bert_P_final if x])
bert_r = compute_bert([x for x in Bert_R_final if x])
bert_f1 = compute_bert([x for x in Bert_F1_final if x])
perplexity = compute_bert([x for x in final_perplexity if x])
val1 = calc_out([x for x in output_len_final if x])
output_len = np.array(val1).mean()

rouge1_boh = compute_average([x for x in rouge_1_boh_final if x])
rouge2_boh = compute_average([x for x in rouge_2_boh_final if x])
rougel_boh = compute_average([x for x in rougle_l_boh_final if x])
bleu_boh = compute_average([x for x in bleu_boh_final if x])
bert_p_boh = compute_bert([x for x in Bert_P_boh_final if x])
bert_r_boh = compute_bert([x for x in Bert_R_boh_final if x])
bert_f1_boh = compute_bert([x for x in Bert_F1_boh_final if x])
perplexity_boh = compute_bert([x for x in final_perplexity_boh if x])
val2 = calc_out([x for x in output_len_boh_final if x])
output_len_boh = np.array(val2).mean()

print("Rouge1: ", rouge1)
print("Rouge2: ", rouge2)
print("RougeL: ", rougel)
print("Bleu: ", bleu)
print("Bert_P: ", bert_p)
print("Bert_R: ", bert_r)
print("Bert_F1: ", bert_f1)
print("Perplexity: ", perplexity)
print("Output Length: ", output_len)

print("Rouge1_boh: ", rouge1_boh)
print("Rouge2_boh: ", rouge2_boh)
print("RougeL_boh: ", rougel_boh)
print("Bleu_boh: ", bleu_boh)
print("Bert_P_boh: ", bert_p_boh)
print("Bert_R_boh: ", bert_r_boh)
print("Bert_F1_boh: ", bert_f1_boh)
print("Perplexity_boh: ", perplexity_boh)
print("Output Length_boh: ", output_len_boh)


Rouge1:  0.11240419841728401
Rouge2:  0.011303157371437297
RougeL:  0.07657559485328606
Bleu:  0.013788763420726689
Bert_P:  0.3605505928476021
Bert_R:  0.46317868024189635
Bert_F1:  0.4025812452856116
Perplexity:  180.8799944287626
Output Length:  2995.25
Rouge1_boh:  0.10986720722140499
Rouge2_boh:  0.007802945650681672
RougeL_boh:  0.0784167312832368
Bleu_boh:  0.011967059944002842
Bert_P_boh:  0.36049814302493965
Bert_R_boh:  0.4593688035590781
Bert_F1_boh:  0.4015508834312044
Perplexity_boh:  454.3965698462407
Output Length_boh:  1874.0


In [10]:
print(final_perplexity)
print(final_perplexity_boh)

[[], [], [], [tensor(2758.9963, device='cuda:0'), tensor(2758.9963, device='cuda:0'), tensor(2758.9963, device='cuda:0'), tensor(255.3050, device='cuda:0'), tensor(255.3050, device='cuda:0'), tensor(83.7973, device='cuda:0'), tensor(83.7973, device='cuda:0'), tensor(44.2446, device='cuda:0'), tensor(44.2446, device='cuda:0'), tensor(44.2446, device='cuda:0'), tensor(30.8414, device='cuda:0'), tensor(30.8414, device='cuda:0'), tensor(30.8414, device='cuda:0'), tensor(24.3132, device='cuda:0'), tensor(24.3132, device='cuda:0'), tensor(20.3720, device='cuda:0'), tensor(20.3720, device='cuda:0'), tensor(17.7163, device='cuda:0'), tensor(17.7163, device='cuda:0'), tensor(15.7338, device='cuda:0'), tensor(15.7338, device='cuda:0'), tensor(14.3006, device='cuda:0'), tensor(14.3006, device='cuda:0')], [], [tensor(647.5068, device='cuda:0'), tensor(55.7980, device='cuda:0'), tensor(55.7980, device='cuda:0'), tensor(55.7980, device='cuda:0'), tensor(55.7980, device='cuda:0'), tensor(22.6013, dev

In [10]:
final_perplexity = [x for x in final_perplexity if x]
final_perplexity_boh = [x for x in final_perplexity_boh if x]

print(final_perplexity)
print(final_perplexity_boh)
print(len(final_perplexity[0]))
print(len(final_perplexity_boh[2]))

[[tensor(2759.0044, device='cuda:0'), tensor(2759.0044, device='cuda:0'), tensor(2759.0044, device='cuda:0'), tensor(255.3056, device='cuda:0'), tensor(255.3056, device='cuda:0'), tensor(83.3565, device='cuda:0'), tensor(83.3565, device='cuda:0'), tensor(43.5371, device='cuda:0'), tensor(43.5371, device='cuda:0'), tensor(43.5371, device='cuda:0'), tensor(29.6003, device='cuda:0'), tensor(29.6003, device='cuda:0'), tensor(29.6003, device='cuda:0'), tensor(22.9927, device='cuda:0'), tensor(22.9927, device='cuda:0'), tensor(19.1316, device='cuda:0'), tensor(19.1316, device='cuda:0'), tensor(16.5980, device='cuda:0'), tensor(16.5980, device='cuda:0'), tensor(14.7532, device='cuda:0'), tensor(14.7532, device='cuda:0'), tensor(13.4343, device='cuda:0'), tensor(13.4343, device='cuda:0')], [tensor(647.5090, device='cuda:0'), tensor(56.0615, device='cuda:0'), tensor(56.0615, device='cuda:0'), tensor(56.0615, device='cuda:0'), tensor(56.0615, device='cuda:0'), tensor(22.9064, device='cuda:0'), t

In [11]:
for i in final_perplexity:
    for j in range(len(i)):
        i[j] = i[j].cpu()
        
for i in final_perplexity_boh:
    for j in range(len(i)):
        i[j] = i[j].cpu()

In [12]:
new_final_perplexity = []
new_final_perplexity_boh = []

for i in range(len(final_perplexity)):
    
    if len(final_perplexity[i]) > 20:
        new_final_perplexity.append(final_perplexity[i][:20])
        new_final_perplexity_boh.append(final_perplexity_boh[i][:20])
        

In [13]:
import numpy as np

perplexity_final_list_filtered = np.array(new_final_perplexity)
perplexity_boh_final_list_filtered = np.array(new_final_perplexity_boh)

print(np.mean(np.mean(perplexity_final_list_filtered, axis=1)))
print(np.mean(np.mean(perplexity_boh_final_list_filtered, axis=1)))

279.50336
731.245


In [14]:
path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/instruct-mistral/evaluate/perplexity_scores/esconv1/"
np.save(path+'llama/llama_base.npy', perplexity_final_list_filtered)
np.save(path+'llama/llama_base_boh.npy', perplexity_boh_final_list_filtered)


In [16]:
print(output_len_final)
print(output_len_boh_final)

[[], [], [], [9, 101, 293, 441, 607, 828, 1019, 1183, 1331, 1524], [], [16, 158, 313, 474, 626, 779, 952, 1103, 1256, 1407, 1553, 1698, 1851, 1991, 2148, 2293, 2447, 2605, 2748, 2888, 3051, 3200, 3347, 3509, 3653, 3797], [9, 49, 125, 195, 293, 440, 581, 730, 874, 1028, 1187, 1349, 1502, 1652, 1795, 1942, 2033, 2231, 2386, 2489, 2668], [], [], [10, 47, 105, 242, 399, 552, 723, 862, 1051, 1208, 1353, 1505, 1655, 1795, 1940, 2093, 2245, 2399, 2541, 2689, 2845, 2993, 3134, 3278, 3430, 3573]]
[[], [], [], [9, 59, 162, 241, 369, 488, 600, 685, 781, 907], [], [13, 105, 186, 291, 390, 490, 576, 673, 769, 867, 971, 1025, 1079, 1141, 1213, 1311, 1412, 1510, 1594, 1704, 1808, 1911, 2011, 2124, 2220, 2331], [9, 33, 68, 124, 210, 297, 393, 492, 579, 681, 789, 926, 1025, 1126, 1236, 1349, 1425, 1551, 1639, 1692, 1804], [], [], [10, 34, 77, 147, 245, 337, 433, 535, 607, 713, 809, 900, 1003, 1101, 1199, 1313, 1410, 1524, 1632, 1744, 1863, 1977, 2085, 2188, 2298, 2392]]


In [15]:
new_output_len = []
new_output_len_boh = []

for i in range(len(output_len_final)):
    
    if len(output_len_final[i]) > 5:
        new_output_len.append(output_len_final[i][:10])
        new_output_len_boh.append(output_len_boh_final[i][:10])

In [16]:
print(new_output_len)
print(new_output_len_boh)

[[9, 101, 293, 442, 665, 885, 1077, 1241, 1389, 1568], [16, 158, 313, 475, 628, 781, 954, 1105, 1258, 1409], [9, 49, 135, 211, 341, 488, 628, 776, 920, 1074], [10, 47, 109, 262, 420, 574, 745, 883, 1072, 1229]]
[[9, 59, 163, 240, 341, 463, 574, 661, 754, 874], [13, 89, 172, 264, 367, 468, 571, 665, 759, 855], [9, 33, 62, 111, 195, 270, 346, 435, 514, 611], [10, 34, 84, 174, 261, 361, 459, 550, 635, 730]]


In [17]:
import numpy as np

new_output_len_filtered = np.array(new_output_len)
new_output_len_boh_filtered = np.array(new_output_len_boh)

print(np.mean(np.mean(new_output_len_filtered, axis=1)))
print(np.mean(np.mean(new_output_len_boh_filtered, axis=1)))    

618.725
356.125


In [20]:

np.save(path+'llama/llama_base_out.npy', new_output_len_filtered)
np.save(path+'llama/llama_base_out_boh.npy', new_output_len_boh_filtered)


In [19]:
print(new_output_len_filtered)

[[   9  101  293  442  665  885 1077 1241 1389 1568]
 [  16  158  313  475  628  781  954 1105 1258 1409]
 [   9   49  135  211  341  488  628  776  920 1074]
 [  10   47  109  262  420  574  745  883 1072 1229]]
